In [1]:
import os
import re
from io import BytesIO

import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from itables import init_notebook_mode
from pypdf import PdfWriter, PdfReader

from urllib.parse import urlsplit

init_notebook_mode(all_interactive=True)

<IPython.core.display.Javascript object>

### Webscraping functions

In [2]:
ua = UserAgent(min_percentage=2.1)
ua.random
headers = {'User-Agent': str(ua.random)}

In [3]:
def is_url(cell):
    return bool(re.match("^https://", cell))

def get_url_content(domain):
    links = []
    if is_url(domain):
        # print(domain)
        headers = {'User-Agent': str(ua.random)}
        response = requests.get(domain, headers=headers)
        # print(response.url)
        soup = BeautifulSoup(response.text, 'lxml')
        if bool(re.match("^https://cgspace.cgiar.org/handle/", response.url)):
            # if there are any pdfs on the page directly access them - https://cgspace.cgiar.org/handle/10568/126321; https://cgspace.cgiar.org/handle/10568/116411; https://cgspace.cgiar.org/handle/10568/121051
            links.extend([a['href'] for a in soup.find_all('a', {'href': re.compile(".*\.pdf$")})])
            # files on Github - ex. https://hdl.handle.net/10568/127746
            if len(links) == 0:
                links.extend(meta['content'] for meta in soup.find_all('meta', {'name': re.compile("citation_(pdf|abstract)_url")}))
            if len(links) == 0:
                hrefs = [a['href'] for a in soup.find_all('a', href=True)]
                for href in hrefs:
                    if href.startswith("https://doi.org/"):
                        response = requests.get(href, headers=headers)
                        url = response.url
                        response = requests.get(url, headers=headers)
                        soup = BeautifulSoup(response.text, 'lxml')
                        if bool(re.match("^https://academic.oup.com/wber/article/", response.url)):
                            links.extend(meta['content'] for meta in soup.find_all('meta', {'name': re.compile("citation_(pdf|abstract)_url")}))
                        elif bool(re.match("^https://link.springer.com/article/", response.url)):
                            links.extend(meta['content'] for meta in soup.find_all('meta', {'name': re.compile("citation_(pdf|abstract)_url")}))
                        elif bool(re.match("^https://gh.bmj.com/content/", response.url)):
                            links.extend(meta['content'] for meta in soup.find_all('meta', {'name': re.compile("citation_(pdf|abstract)_url")}))
                        # elif bool(re.match("^https://linkinghub.elsevier.com/retrieve/pii/", response.url)):
                        #     print("Science Direct")
                        #     _url = response.url
                        #     id = _url.split("/")[-1]
                        #     response = requests.get(url = f"https://www.sciencedirect.com/science/article/pii/{id}", headers=headers)
                        #     anchor = soup.find('a', {'href': lambda x: x.startswith('https://www.sciencedirect.com/science/article/pii/')})
                        #     print(anchor)
                        #     # doc_id =
                        #     # links.extend([f"https://www.sciencedirect.com/science/article/pii/{doc_id}.pdf"])
                    elif href.startswith("https://www.iwmi.cgiar.org/"):
                        links.extend([href])
                    else:
                        continue
        else:
            print(f"Error: {response.url}. Unable to find a PDF file link.")
        if len(links) > 0:
            return "".join(links)
        else:
            return pd.NA

# still in development - if https://doi.org/ redirects to https://www.sciencedirect.com/ then scrape https://www.sciencedirect.com/[...]-main.pdf - https://doi.org/10.1016/j.ehb.2022.101185, https://doi.org/10.1016/j.pce.2021.103082
# still in development - if https://doi.org/ redirects to https://www.cambridge.org/core/journals/ then scrape https://www.cambridge.org/core/services/aop-cambridge-core/content/view/[...].pdf

### 2022 Innovation Development Document

In [258]:
df = pd.read_excel("/Users/.../file_1698742987140.xlsx")

The original file contain 477 records.

#### Get direct URL to evidence and save to updated document

In [260]:
df["Evidence 1"] = df["Evidence 1"].astype(str)
df["Evidence 2"] = df["Evidence 2"].astype(str)
df["Evidence 3"] = df["Evidence 3"].astype(str)

In [261]:
df_filtered = df[df['Evidence 1'].apply(is_url) | df['Evidence 2'].apply(is_url) | df['Evidence 3'].apply(is_url)]

Only 126 of these records contains at least 1 link for one of the three fields: 'Evidence 1', 'Evidence 2', 'Evidence 3'.

In [262]:
df_filtered

Result id                                           PDF link  \
2         1099  https://reporting.cgiar.org/reports/result-det...   
15        1036  https://reporting.cgiar.org/reports/result-det...   
21        2897  https://reporting.cgiar.org/reports/result-det...   
28        2793  https://reporting.cgiar.org/reports/result-det...   
29        1031  https://reporting.cgiar.org/reports/result-det...   
..         ...                                                ...   
460       1593  https://reporting.cgiar.org/reports/result-det...   
461       1489  https://reporting.cgiar.org/reports/result-det...   
465       1026  https://reporting.cgiar.org/reports/result-det...   
472       1100  https://reporting.cgiar.org/reports/result-det...   
474        805  https://reporting.cgiar.org/reports/result-det...   

    Primary submitter              Level                    Type  \
2             INIT-17  Initiative output  Innovation development   
15            INIT-34  Initiative output  Innovation development   
21            INIT-22  Initiative output  Innovation development   
28            INIT-03  Initiative output  Innovation development   
29            INIT-34  Initiative output  Innovation development   
..                ...                ...                     ...   
460           INIT-25  Initiative output  Innovation development   
461           INIT-25  Initiative output  Innovation development   
465           INIT-28  Initiative output  Innovation development   
472           INIT-34  Initiative output  Innovation development   
474           INIT-21  Initiative output  Innovation development   

                                                 Title  \
2    Integrated-approach with Public Private Partne...   
15   An improved framework for cost benefit analysi...   
21   Vitamin A rich banana adoption in Cibitoke and...   
28   Conservation methods developed for rare wild b...   
29   Agri-climate risk scoring system for livestock...   
..                                                 ...   
460  High-frequency diet quality data collection an...   
461  Biomass Climate Adaptation Index (CAI): A sate...   
465  New Water Storage Diagnostic to diversify and ...   
472  Multidsciplinary working group (MWG) as a dedi...   
474  Through this initiative, farmers receive key c...   

                                           Description  \
2    "In Mali, vaccination campaigns against major ...   
15   A framework(s) for improving large-scale land ...   
21   A positive perception was observed among farme...   
28   A rare bean species, Phaseolus hygrophilus, wh...   
29   A satellite-based agri-climate risk scoring sy...   
..                                                 ...   
460  CGIAR Research Initiative on Digital Innovatio...   
461  The lack of systematic tools and approaches fo...   
465  The Water Storage Diagnostic is an approach to...   
472  The multidisciplinary working group (MWG) is a...   
474  Shamba shape is a farm makeover show which rea...   

                                   Lead contact person      Gender level  \
2                                         Michel Dione   (1) Significant   
15                   Fiona Flintan f.flintan@cgiar.org  (0) Not targeted   
21   Walter Ocimati, w.ocimati@cgiar.org, 46-Allian...  (0) Not targeted   
28                                         Peter Wenzl  (0) Not targeted   
29   Shalika Yvas, s.vyas@cgiar.org, Alliance Biove...   (1) Significant   
..                                                 ...               ...   
460                                 Rhys Manners, IITA   (1) Significant   
461  Louis Reymondin, The Alliance of Bioversity In...  (0) Not targeted   
465                                    Jonathan Lautze  (0) Not targeted   
472                                     Issa Ouedraogo  (0) Not targeted   
474                                     Pedro Chilambe   (1) Significant   

    Climate change level  ... Is new variety?  numbe

#### Test Webscraping functions

In [5]:
cell = "https://cgspace.cgiar.org/handle/10568/121051"

output = get_url_content(cell)
print(output)

https://www.iwmi.cgiar.org/tools/sip-sizing-tool/sip_sizing_manual.pdf


In [54]:
url = "https://cgspace.cgiar.org/bitstream/10568/127891/2/CBA_Silestri.pdf"
headers = {'User-Agent': str(ua.random)}
response = requests.get(url, headers=headers)

print(response.status_code)

if response.status_code == 200:
    pdf_data = response.content
    with open("/Users/.../data/2897/CBA_Silestri.pdf", "wb") as f:
        f.write(pdf_data)
else:
    print("Error: Unable to download PDF file.")

200


In [82]:
response = requests.get('https://api.reporting.cgiar.org/api/platform-report/result/32?phase=1')

if response.status_code == 200:
    # Read the PDF data from the response content
    pdf_reader = PdfReader(BytesIO(response.content))

    with open("/Users/.../2022/32/result.pdf", "wb") as f:
        pdf_writer = PdfWriter()
        for page in pdf_reader.pages:
            pdf_writer.add_page(page)
        pdf_writer.write(f)

#### Get direct URL to evidence and save to updated document

In [290]:
df_updated = df_filtered.copy()
df_updated.loc[:, 'Evidence 1 Links'] = df_filtered['Evidence 1'].apply(get_url_content)
df_updated.loc[:, 'Evidence 2 Links'] = df_filtered['Evidence 2'].apply(get_url_content)
df_updated.loc[:, 'Evidence 3 Links'] = df_filtered['Evidence 3'].apply(get_url_content)

/Users/ebenge/mambaforge/envs/ai/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [291]:
df_updated.to_excel("/Users/.../evidence_links_file_1698742987140_v2.xlsx")

#### Read in updated 2022 Innovation Development Document

In [6]:
df_updated = pd.read_excel("/Users/.../evidence_links_file_1698742987140_v2.xlsx")

In [7]:
print(df_updated.columns)

Index(['Unnamed: 0', 'Result id', 'PDF link', 'Primary submitter', 'Level',
       'Type', 'Title', 'Description', 'Lead contact person', 'Gender level',
       'Climate change level', 'Actors', 'Is KRS', 'KRS link',
       'From previous portfolio', 'Legacy ID', 'Contributing initiatives',
       'Non pooled projects', 'Primary center', 'Contributing centers',
       'TOC results', 'Action areas', 'Impact areas', 'toc_sdgs', 'Partners',
       'Countries', 'Regions', 'Linked results', 'Short title',
       'Innovation characterization', 'Innovation nature', 'Is new variety?',
       'number_of_varieties', 'Developers', 'Collaborators', 'Acknowledgement',
       'Readiness level', 'Evidences explanation', 'Evidence 1', 'Evidence 2',
       'Evidence 3', 'Evidence 1 Links', 'Evidence 2 Links',
       'Evidence 3 Links'],
      dtype='object')


### Download Result PDFs

In [267]:
df_view = df_updated.loc[:, ['Result id', 'PDF link']]
df_view

Result id                                           PDF link
0         1099  https://reporting.cgiar.org/reports/result-det...
1         1036  https://reporting.cgiar.org/reports/result-det...
2         2897  https://reporting.cgiar.org/reports/result-det...
3         2793  https://reporting.cgiar.org/reports/result-det...
4         1031  https://reporting.cgiar.org/reports/result-det...
..         ...                                                ...
121       1593  https://reporting.cgiar.org/reports/result-det...
122       1489  https://reporting.cgiar.org/reports/result-det...
123       1026  https://reporting.cgiar.org/reports/result-det...
124       1100  https://reporting.cgiar.org/reports/result-det...
125        805  https://reporting.cgiar.org/reports/result-det...

[126 rows x 2 columns]

In [13]:
root = "/Users/.../2022/"

for index, row in df_view.iterrows():
    id = row['Result id']

    path = f"{root}/{id}"
    if not os.path.exists(path):
        os.makedirs(path)

    if id not in (754, 999, 1783, 28, 1003, 35, 455) :  # results not available on PRMS
        response = requests.get(f"https://api.reporting.cgiar.org/api/platform-report/result/{id}?phase=1")

        if response.status_code == 200:
            pdf_reader = PdfReader(BytesIO(response.content))

            with open(f"/Users/.../2022/{id}/result.pdf", "wb") as f:
                pdf_writer = PdfWriter()
                for page in pdf_reader.pages:
                    pdf_writer.add_page(page)
                pdf_writer.write(f)
            print(f"Successfully downloaded result.pdf for Result ID {id}")
        else:
            print(f"Error: Unable to download result.pdf for Result ID {id}")

Successfully downloaded result.pdf for Result ID 1099
Successfully downloaded result.pdf for Result ID 1036
Successfully downloaded result.pdf for Result ID 2897
Successfully downloaded result.pdf for Result ID 2793
Successfully downloaded result.pdf for Result ID 1031
Successfully downloaded result.pdf for Result ID 2050
Successfully downloaded result.pdf for Result ID 2051
Successfully downloaded result.pdf for Result ID 3352
Successfully downloaded result.pdf for Result ID 1933
Successfully downloaded result.pdf for Result ID 2049
Successfully downloaded result.pdf for Result ID 4071
Successfully downloaded result.pdf for Result ID 952
Successfully downloaded result.pdf for Result ID 4070
Successfully downloaded result.pdf for Result ID 1112
Successfully downloaded result.pdf for Result ID 789
Successfully downloaded result.pdf for Result ID 1283
Successfully downloaded result.pdf for Result ID 345
Successfully downloaded result.pdf for Result ID 247
Successfully downloaded result.p

### Download Evidence PDFs

In [14]:
def get_filename(link):
    return os.path.splitext(os.path.basename(urlsplit(link)[2]))[0]

def download_evidence(root, row):
    exclusion = ['https://cgspace.cgiar.org/bitstream/10568/126881/1/Reporte_CETC-IL_Intibuc%c3%a1_2022.pdf']
    links = []
    id = row['Result id']
    path = f"{root}/{id}"
    if not os.path.exists(path):
        os.makedirs(path)
    for col in ['Evidence 1 Links', 'Evidence 2 Links', 'Evidence 3 Links']:
        if isinstance(row[col], str) and row[col] not in exclusion:
            links.append(row[col])
    for link in links:
        print("url: ", link)
        fname = get_filename(link)
        response = requests.get(link, headers=headers)
        print("response code: ", response.status_code)
        if response.status_code == 200:
            data = response.content
            if os.path.splitext(os.path.basename(urlsplit(link)[2]))[0] == ".pdf":
                with open(f"{path}/{fname}.pdf", 'wb') as f:
                    f.write(data)
            elif os.path.splitext(os.path.basename(urlsplit(link)[2]))[0] == ".pptx":
                with open(f"{path}/{fname}.pptx", 'wb') as f:
                    f.write(data)
        else:
            print(f"Error: Unable to download {fname}.pdf")

In [270]:
print(os.path.splitext(os.path.basename(urlsplit("https://cgspace.cgiar.org/bitstream/10568/127872/1/AAEA_aug_022.pptx")[2]))[1])

.pptx


In [15]:
root = "/Users/.../2022"
df_updated.apply(lambda x: download_evidence(root, x), axis=1)

url:  https://cgspace.cgiar.org/bitstream/10568/127891/2/CBA_Silestri.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/127269/1/20-12-22%20Burundi%20pVAB%20adoption%20study%20report%20ABC_TAFS%20WCA.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/121675/1/OutcomesCaseStudySummary-CCAFS-P259-OICS2107.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/120464/1/report.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/128475/2/247c5df1711838df9983c409afe9d715.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/128080/1/65942.pdf
response code:  200
url:  https://academic.oup.com/wber/article-pdf/36/4/889/46673080/lhac015.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/128124/2/59f8c07c55a4552db8db85044cfebee6.pdf
response code:  200
url:  https://cgspace.cgiar.org/bitstream/10568/125752/2/56284a8353715c3dcf456dc5bada447f.pdf
response code:  200
url:  https:/

0
Loading... (need help?)
